# Homework 1: Preprocessing and Text Classification

Student Name: Yang Xiong

Student ID: 722890

Python version used: 3.6.4

## General info

<b>Due date</b>: 11pm, Sunday March 18th

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be using a corpus of tweets to do tokenisation of hashtags and build polarity classifers using bag of word (BOW) features.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks.  

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a few minutes, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## Preprocessing

<b>Instructions</b>: For this homework we will be using the tweets in the <i>twitter_samples</i> corpus included with NLTK. You should start by accessing these tweets. Use the <i>strings</i> method included in the NLTK corpus reader for <i>twitter_samples</i> to access the tweets (as raw strings). Iterate over the full corpus, and print out the average length, in characters, of the tweets in the corpus. (0.5)


In [ ]:
from nltk.corpus import twitter_samples
sum([len(t) for t in twitter_samples.strings()])/float(len(twitter_samples.strings()))

<b>Instructions</b>: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. You should use a regular expression to extract all hashtags of length 8 or longer which consist only of lower case letters (other than the # at the beginning, of course, though this should be stripped off as part of the extraction process). Do <b>not</b> tokenise the entire tweet as part of this process. The hashtag might occur at the beginning or the end of the tweet; you should double-check that you aren't missing any. After you have collected them into a list, print out number of hashtags you have collected: for full credit, you must get the exact number that we expect.  (1.0)

In [ ]:
import re
pattern = re.compile(r'(?<=\s#)[a-z]{8,}(?=\s|$)|(?<=^#)[a-z]{8,}(?=\s|$)')
sum([len(re.findall(pattern,t)) for t in twitter_samples.strings()])

<b>Instructions</b>: Now, tokenise the hashtags you've collected. To do this, you should implement a reversed version of the MaxMatch algorithm discussed in class (and in the reading), where matching begins at the end of the hashtag and progresses backwards. NLTK has a list of words that you can use for matching, see starter code below. Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatiser before matching. Note that the list of words is incomplete, and, if you are unable to make any longer match, your code should default to matching a single letter. Create a new list of tokenised hashtags (this should be a list of lists of strings) and use slicing to print out the last 20 hashtags in the list. (1.0)

In [ ]:
import nltk
import re
from nltk.corpus import twitter_samples
words = nltk.corpus.words.words() # words is a Python list

# lemmatizer 
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

# MaxMatch algorithm
def max_match(hashtag):
    tokens = []
    index = len(hashtag)
    while len(hashtag) > 0:
        token = lemmatize(hashtag[:index])
        if token in words:
            tokens.append(hashtag[:index])
            hashtag = hashtag[index:]
            index = len(hashtag)
        else:
            index -= 1
    return tokens

# hashtag lists
pattern = re.compile(r'(?<=\s#)[a-z]{8,}(?=\s|$)|(?<=^#)[a-z]{8,}(?=\s|$)')
hashtags = [re.findall(pattern,t) for t in twitter_samples.strings()]

# filter the empty string 
hashtags_filter = [h for h in  hashtags if h]

# get a list of hashtag
hashtag_list =[]
for hs in hashtags_filter:
    for h in hs:
        hashtag_list.append(h)

result = [max_match(hashtag) for hashtag in hashtag_list]
print(result[-20:])



### Extra Credit (Optional)
<b>Instructions</b>: Implement the forward version of the MaxMatch algorithm as well, and print out all the hashtags which give different results for the two versions of MaxMatch. Your main task is to come up with a good way to select which of the two segmentations is better for any given case, and demonstrate that it works significantly better than using a single version of the algorithm for all hashtags. (1.0)

## Text classification (Not Optional)

<b>Instructions</b>: The twitter_sample corpus has two subcorpora corresponding to positive and negative tweets. You can access already tokenised versions using the <i> tokenized </i> method, as given in the code sample below. Iterate through these two corpora and build training, development, and test sets for use with Scikit-learn. You should exclude stopwords (from the built-in NLTK list) and tokens with non-alphabetic characters (this is very important you do this because emoticons were used to build the corpus, if you don't remove them performance will be artificially high). You should randomly split each subcorpus, using 80% of the tweets for training, 10% for development, and 10% for testing; make sure you do this <b>before</b> combining the tweets from the positive/negative subcorpora, so that the sets are <i>stratified</i>, i.e. the exact ratio of positive and negative tweets is preserved across the three sets. (1.0)

In [ ]:
import re
import nltk
from random import shuffle
from nltk.corpus import stopwords

positive_tweets = nltk.corpus.twitter_samples.tokenized("positive_tweets.json")
negative_tweets = nltk.corpus.twitter_samples.tokenized("negative_tweets.json")
stop_words = set(stopwords.words('english')) 

# function to remove stopwords
def remove_stopwords(text):
    out = []
    for word in text:
        if word not in stop_words:
            out.append(word)
    return out

# function to remove non_alphabetic characters
def remove_nonalpha(text):
    out = []
    for word in text:
        word = re.sub('[^A-Za-z]', '', word)
        if word:
            out.append(word)
    return out

pos_tweets_no_stopwords = [remove_stopwords(t) for t  in positive_tweets ]
neg_tweets_no_stopwords = [remove_stopwords(t) for t  in negative_tweets ]   
pos_tweets_processed = [remove_nonalpha(t) for t in pos_tweets_no_stopwords]
neg_tweets_processed = [remove_nonalpha(t) for t in neg_tweets_no_stopwords]

# shuffle tweeters then spilt 
shuffle(pos_tweets_processed)
shuffle(neg_tweets_processed)


train_pos_tweets = pos_tweets_processed[:4000]
dev_pos_tweets = pos_tweets_processed[4000:4500]
test_pos_tweets = pos_tweets_processed[4500:]

train_neg_tweets = neg_tweets_processed[:4000]
dev_neg_tweets = neg_tweets_processed[4000:4500]
test_neg_tweets = neg_tweets_processed[4500:]

train_tweets = train_pos_tweets + train_neg_tweets
dev_tweets = dev_pos_tweets + dev_neg_tweets
test_tweets = test_pos_tweets + test_neg_tweets


<b>Instructions</b>: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation (hyper)parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do <b>not</b> use crossvalidation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the accuracy with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output. (1.0)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report,accuracy_score

def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW

def prepare_reuters_data(tweets, attitude, feature_extractor):
    feature_matrix = []
    classifications = []
    for tweet in tweets:
        feature_dict = feature_extractor(tweet)
        feature_matrix.append(feature_dict)
        classifications.append(attitude)        
    return feature_matrix, classifications

tweets_pos, classifications_pos = prepare_reuters_data(pos_tweets_processed, 'positive', get_BOW)
tweets_neg, classifications_neg = prepare_reuters_data(neg_tweets_processed, 'negative', get_BOW)

train_tweets = tweets_pos[:4000] + tweets_neg[:4000]
dev_tweets = tweets_pos[4000:4500] + tweets_neg[4000:4500]
test_tweets = tweets_pos[4500:] + tweets_neg[4500:]

train_tweets_label = classifications_pos[:4000] + classifications_neg[:4000]
dev_tweets_label = classifications_pos[4000:4500] + classifications_neg[4000:4500]
test_tweets_label = classifications_pos[4500:] + classifications_neg[4500:]

#print(train_tweets_label[3999:4001])

#create the transform 
vectorizer = DictVectorizer()

vectorizer.fit(train_tweets)
train_data = vectorizer.transform(train_tweets)
dev_data = vectorizer.transform(dev_tweets)
test_data = vectorizer.transform(test_tweets)
#train_data._shape

# default setting
nb_clf_default = MultinomialNB()
nb_clf_default.__init__(alpha= 1.0, fit_prior=True, class_prior=None)

nb_clf_default.fit(train_data, train_tweets_label)
dev_tweets_predict_nb = nb_clf_default.predict(dev_data)
print("Naive Bayes accuracy with default setting alpha(1.0)" )
print(accuracy_score(dev_tweets_label,dev_tweets_predict_nb))
#print (classification_report(dev_tweets_label, dev_tweets_predict))

# small alpha value, alpha = 0.01
nb_clf_small_alpha = MultinomialNB()
nb_clf_small_alpha.__init__(alpha= 0.01, fit_prior=True, class_prior=None)

nb_clf_small_alpha.fit(train_data, train_tweets_label)
dev_tweets_predict_nb = nb_clf_small_alpha.predict(dev_data)
print("Naive Bayes accuracy with small alpha(0.01)")
print(accuracy_score(dev_tweets_label,dev_tweets_predict_nb))
#print (classification_report(dev_tweets_label, dev_tweets_predict))

# big alpha value, alpha = 4
nb_clf_big_alpha = MultinomialNB()
nb_clf_big_alpha.__init__(alpha= 4, fit_prior=True, class_prior=None)

nb_clf_big_alpha.fit(train_data, train_tweets_label)
dev_tweets_predict_nb = nb_clf_big_alpha.predict(dev_data)
print("Naive Bayes accuracy with big alpha(4)")
print(accuracy_score(dev_tweets_label,dev_tweets_predict_nb))
#print (classification_report(dev_tweets_label, dev_tweets_predict))

# tuned setting, alpha = 0.9
nb_clf_tuned = MultinomialNB()
nb_clf_tuned.__init__(alpha= 0.9, fit_prior=True, class_prior=None)
nb_clf_tuned.fit(train_data, train_tweets_label)
dev_tweets_predict_nb = nb_clf_tuned.predict(dev_data)
print("Naive Bayes accuracy with near optimal setting, alpha(0.9)")
print(accuracy_score(dev_tweets_label,dev_tweets_predict_nb))
print("")
print("According to above results, the near optimal choice for Naive Bayes is alpha = 0.9")
print("")


# default setting: l2 penalty c = 1.0 
lr_clf_default = LogisticRegression()
lr_clf_default.fit(train_data, train_tweets_label)
dev_tweets_predict_lr = lr_clf_default.predict(dev_data)
print("Logistic Regression accuracy with default setting, l2 penalty c = 1.0: ")
print(accuracy_score(dev_tweets_label,dev_tweets_predict_lr))
#print (classification_report(dev_tweets_label, dev_tweets_predict))

# l2 penalty c = 100.0
lr_clf_l2_bc = LogisticRegression()
lr_clf_l2_bc.__init__(C=100.0)
lr_clf_l2_bc.fit(train_data, train_tweets_label)
dev_tweets_predict_lr = lr_clf_l2_bc.predict(dev_data)
print("Logistic Regression accuracy with l2 penalty, C=100.0: ")
print(accuracy_score(dev_tweets_label,dev_tweets_predict_lr))

# l2 penalty c = 0.1 if c<1.0 then the accuracy score starts to 
# decrease 
lr_clf_l2_sc = LogisticRegression()
lr_clf_l2_sc.__init__(C=0.1)
lr_clf_l2_sc.fit(train_data, train_tweets_label)
dev_tweets_predict_lr = lr_clf_l2_sc.predict(dev_data)
print("Logistic Regression accuracy with l2 penalty, C=0.1: ")
print(accuracy_score(dev_tweets_label,dev_tweets_predict_lr))

# apply l1 penalty c = 1.0
lr_clf_l1 = LogisticRegression()
lr_clf_l1.__init__(penalty='l1',C=1.0)
lr_clf_l1.fit(train_data, train_tweets_label)
dev_tweets_predict_lr = lr_clf_l1.predict(dev_data)
print("Logistic Regression accuracy with l1 penalty, C=1.0: ")
print(accuracy_score(dev_tweets_label,dev_tweets_predict_lr))

# apply l1 penalty C = 5.0, with the increase of C, accuracy raise to peak when C is
# around 5.0, when the C greater then 5.0, the accuracy starts to decrease
lr_clf_l1_bc = LogisticRegression()
lr_clf_l1_bc.__init__(penalty='l1',C=100.0)
lr_clf_l1_bc.fit(train_data, train_tweets_label)
dev_tweets_predict_lr = lr_clf_l1_bc.predict(dev_data)
print("Logistic Regression accuracy with l1 penalty, C =100.0: ")
print(accuracy_score(dev_tweets_label,dev_tweets_predict_lr))

# apply l1 penalty C = 0.1, with the decrease of C, the accuracy starts to decrease
# with all settings of l1 penalty, all accuracy scores are smaller than after applying
# l2 penalty 
lr_clf_l1_sc = LogisticRegression()
lr_clf_l1_sc.__init__(penalty='l1',C=0.1)
lr_clf_l1_sc.fit(train_data, train_tweets_label)
dev_tweets_predict_lr = lr_clf_l1_sc.predict(dev_data)
print("Logistic Regression accuracy with l1 penalty, C=0.1: ")
print(accuracy_score(dev_tweets_label,dev_tweets_predict_lr))
print("")
print("According to above results, the near optimal choice for Logistic Regression is c = 1.0, penalty = l2")

<b>Instructions</b>: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both accuracy and macroaveraged f-score for each classifier. Be sure to label your output. (0.5)

In [ ]:
from sklearn.metrics import f1_score

# For Naive Bayes, the best setting is alpha = 0.9 
test_tweets_predict_nb = nb_clf_tuned.predict(test_data)
print("Naive Bayes accuracy: ")
print(accuracy_score(test_tweets_label,test_tweets_predict_nb))
print("Naive Bayes macroaveraged f-score")
print(f1_score(test_tweets_label,test_tweets_predict_nb, average='macro'))

print("")

# For Logistic Regression, the best setting is applying L2 penalty and c = 1.0
test_tweets_predict_lr = lr_clf_default.predict(test_data)
print("Logistic Regression accuracy")
print(accuracy_score(test_tweets_label,test_tweets_predict_lr))
print("Logistic Regression macroaveraged f-score")
print(f1_score(test_tweets_label,test_tweets_predict_lr, average='macro'))


In [ ]:
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import classification_report

predictions = []
for sent in test_corpus:
    in_list = [t[0] for t in sent]
    pred_sent = viterbi((pi, A, O), in_list)
    predictions.append(pred_sent)

new_pred_tags = []
for sent in predictions:
    output = get_tag(sent)
    new_pred_tags.append(output)

all_new_pred_tags = [tag for tags in new_pred_tags for tag in tags]
preci
print (acc(all_test_tags, all_new_pred_tags))

